# Criação do DataSet e Análise das Rotulações

## Pacotes

In [1]:
import pandas as pd
import numpy as np
# Importando a biblioteca utilsDS que possuí funções auxiliares compartilhadas
# com outros notebooks, exemplo: jsonl2list, merge, tokenizeAn
from utilsDS import *

In [13]:
DATA_PATH = "./dados/"

## Lendo e juntando rotulações de todos rotuladores 

In [42]:
## Código para fazer leitura dos arquivos rotulado e não rotulados, realizar o merge entre eles
## e tokenizar as informações. Apenas Thiago e Arthur

# Dados Arthur
# Junta dados rotulados e não rotulados
lista_ArthurCompleta = merge(jsonl2list(DATA_PATH + "arthur_unknown.jsonl"), jsonl2list(DATA_PATH + "arthur.jsonl"))
dadosArthurFranco, referencia = tokenizeAn(lista_ArthurCompleta)

# Dados Thiago                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# Junta dados rotulados e não rotulados
lista_ThiagoCompleta = merge(jsonl2list(DATA_PATH + "thiago_unknown.jsonl"), jsonl2list(DATA_PATH + "thiago.jsonl"))
dadosThiagoSalles = tokenizeAn(lista_ThiagoCompleta)[0]

Leitura e merge dos dados da priscilla

In [15]:
# Dados Priscilla
# Junta dados rotulados e não rotulados
lista_PriscillaCompleta = merge2(jsonl2list(DATA_PATH + "priscilla_unknown.jsonl"), jsonl2list(DATA_PATH + "priscilla.jsonl"), lista_ArthurCompleta)

## Tokenizar os dados da Priscilla
dadosPriscillSilva = tokenizeAn(lista_PriscillaCompleta)[0]

# Criando DataSet no formato IOB

In [16]:
# Tokeniza lista de dados de cada rotuladore
# gerando uma lista com tokens, rotulo e senteça
dsArthur = makingTheDataset(lista_ArthurCompleta)
dsThiago = makingTheDataset(lista_ThiagoCompleta)
dsPriscilla = makingTheDataset(lista_PriscillaCompleta)

### Gerando csv com o dataset IOB de cada rotulador
- Estes DataSets foram utilizados como referencia para resolução das divergencias.

In [17]:
df = pd.DataFrame(dsArthur, columns = ['Palavras', 'Rotulo', 'Sentenca'])
gerarCSV(DATA_PATH + "dsArthur", df)
df = pd.DataFrame(dsThiago, columns = ['Palavras', 'Rotulo', 'Sentenca'])
gerarCSV(DATA_PATH + "dsThiago", df)
df = pd.DataFrame(dsPriscilla, columns = ['Palavras', 'Rotulo', 'Sentenca'])
gerarCSV(DATA_PATH + "dsPriscilla", df)

### Contabilizando divergencias entre os rotuladores

In [18]:
dados = zip(dsArthur, dsThiago, dsPriscilla)

divergenciaPriscilla = 0
divergenciaArthur = 0
divergenciaThiago = 0
divergenciaEntreOsTres = 0

for dadoArthur, dadoThiago, dadoPriscilla in dados:
    if dadoArthur[1] == dadoThiago[1] and dadoArthur[1] == dadoPriscilla[1]: ## Todos sendos iguais
        pass
    elif dadoArthur[1] == dadoThiago[1] and dadoArthur[1] != dadoPriscilla[1]: ## Priscilla divergindo
        divergenciaPriscilla += 1
    elif dadoArthur[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Thiago divergindo
        divergenciaThiago += 1
    elif dadoThiago[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Arthur divergindo
        divergenciaArthur += 1
    else:
        divergenciaEntreOsTres += 1

print(f'divergenciaPriscilla: {divergenciaPriscilla}\ndivergenciaArthur: {divergenciaArthur}\ndivergenciaThiago: {divergenciaThiago}\ndivergenciaEntreOsTres: {divergenciaEntreOsTres}')

divergenciaPriscilla: 3706
divergenciaArthur: 1846
divergenciaThiago: 3020
divergenciaEntreOsTres: 403


### Gerando DataSet para analise das divergências entre rotuladores

In [19]:
dadosFinais = []
textos = [i['data'] for i in lista_ArthurCompleta]
dados = zip(dsArthur, dsThiago, dsPriscilla)

# todas sentecas com discordancia entre os 3 rotuladores
for dadoArthur, dadoThiago, dadoPriscilla in dados:
    if dadoArthur[1] == dadoThiago[1] and dadoArthur[1] == dadoPriscilla[1]: ## Todos sendos iguais
        dadoArthur.append('[N]')
        dadosFinais.append(dadoArthur)
    elif dadoArthur[1] == dadoThiago[1] and dadoArthur[1] != dadoPriscilla[1]: ## Priscilla divergindo
        dadoArthur.append('[N]')
        dadosFinais.append(dadoArthur)
    elif dadoArthur[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Thiago divergindo
        dadoArthur.append('[N]')
        dadosFinais.append(dadoArthur)
    elif dadoThiago[1] == dadoPriscilla[1] and dadoArthur[1] != dadoThiago[1]: ## Arthur divergindo
        dadoThiago.append('[N]')
        dadosFinais.append(dadoThiago)
    else:
        dadoThiago.append('[S]')
        dadosFinais.append(dadoThiago)
        

In [21]:
# Dataset no formato IOB com informação sobre onde houve divergência entre os três rotuladores
# Divergencia sinalizada por [S] na coluna 'Divergencia'
df = pd.DataFrame(dadosFinais, columns = ['Palavras', 'Rotulo', 'Sentenca','Divergencia'])
gerarCSV(DATA_PATH + "Dataset_IOB_Divergencias", df)

## Adicionando posição de Inicio e Fim para cada token no dataset em relação a sentença que ele faz parte

- Utilizamos o dataset "Dataset_IOB_Divergencias_Corrigidas.csv", dataset gerado na etapa anterior após correções feitas pelos tres rotuladores.

In [23]:
dados, referencia = tokenizeAn(lista_ArthurCompleta)
finalDS = pd.read_csv(DATA_PATH + "Dataset_IOB_Divergencias_Corrigidas.csv", usecols=['Palavras', 'Rotulo', 'Sentenca'])


In [24]:
positions = [x[2] for x in referencia]
positions = pd.DataFrame(positions, columns = ['Inicio','Fim'])


In [25]:
start = positions['Inicio']
end = positions['Fim']

In [26]:
finalDS = finalDS.join(start)
finalDS = finalDS.join(end)
finalDS

,Palavras,Rotulo,Sentenca,Inicio,Fim
0,id,O,1,0,2
1,:,O,1,2,3
2,0,O,1,4,5
3,NOME,O,2,0,4
4,DA,O,2,5,7
...,...,...,...,...,...
186014,Destilados,O,14287,54,64
186015,do,O,14287,65,67
186016,Brasil,B-NOME_LOCAL,14287,68,74
186017,2019,B-TEMPO,14287,75,79


In [27]:
finalDS.to_csv(DATA_PATH + "FinalDataSet.csv", encoding="utf-8")

## Atualizando Inicio e Fim para cada token no dataset em relação a sentença que ele faz parte
- após alteração manual no dataset foi preciso atualizar a posição de inicio e fim dos tokens na sentença.

In [28]:
# reading sentences
with open("../../DataSets/Dados_Utilizados_Para_RotulacaoManual.txt") as file:
    fixed_sentencas = [line.rstrip() for line in file]
print(len(fixed_sentencas))

14628


In [30]:
# lendo dataset na qual a alteração será feita
finalDS = pd.read_csv("../../DataSets/dataset_dados_rotulados.csv", usecols=['Palavras', 'Rotulo', 'Sentenca'])
len(finalDS)

186019

In [31]:
# token's position's in the setence
positions = []
for sent in fixed_sentencas:
    wordSpanList = list(TreebankWordTokenizer().span_tokenize(sent))
    for wordSpan in wordSpanList:
        # posicoes
        positions.append([wordSpan[0], wordSpan[1]])
len(positions)

186019

In [32]:
positions = pd.DataFrame(positions, columns = ['Inicio','Fim'])
start = positions['Inicio']
end = positions['Fim']

In [33]:
finalDS = finalDS.join(start)
finalDS = finalDS.join(end)
finalDS

,Palavras,Rotulo,Sentenca,Inicio,Fim
0,id,O,1,0,2
1,:,O,1,2,3
2,0,O,1,4,5
3,NOME,O,2,0,4
4,DA,O,2,5,7
...,...,...,...,...,...
186014,Destilados,O,14628,54,64
186015,do,O,14628,65,67
186016,Brasil,B-NOME_LOCAL,14628,68,74
186017,2019,B-TEMPO,14628,75,79


In [34]:
finalDS.to_csv("../../DataSets/Dataset_Padrao_IOB.csv", encoding="utf-8")

## Transforma DataSet para formato Spacy

In [11]:
finalDS = pd.read_csv("/home/arthur/Documentos/IC/git/ProjetoMestrado/DataSets/FinalDataSetWith_InitialAndFinalPositionOfWordsInPhrase .csv", usecols=['Palavras', 'Rotulo', 'Sentenca', 'Inicio', 'Fim'])

## To SpaCy

In [37]:
# TODO - Ler textos.
with open("../../DataSets/Dados_Utilizados_Para_RotulacaoManual.txt", "r") as file:
    textos = file.read().splitlines()

len(textos)


14628

In [ ]:
sMarker = 0

s = textos[sMarker]
TRAIN_DATA = []
entities = []
for index, row in finalDS.iterrows():
    
    if(row['Sentenca']-1 != sMarker): # Nova sentença
        if entities:
            TRAIN_DATA.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        entities = [] # esvazia entidades
        sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
        if (sMarker < len(textos)): # Limite de textos
            s = textos[sMarker]

    if(row['Rotulo'][0] == 'B'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    if(row['Rotulo'][0] == 'O'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    if(row['Rotulo'][0] == 'I'):
        if (entities): 
            entities[-1][1] = row['Fim']
        else:
            # print(index)
            entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    
    if (index == finalDS.index[-1]): # Ultimo elemento
        if entities:
            TRAIN_DATA.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        
        entities = [] 


TRAIN_DATA

In [47]:
# Usar pickle pra gerar .bin
# print(type(TRAIN_DATA[0][1]['entities']))
cont = 0
for data in TRAIN_DATA:
    if( not data[1]['entities']):
        cont = cont + 1

print(cont)

0
